In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# Creamos una clase del modelo
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim) #https://github.com/pytorch/pytorch/blob/v2.6.0/torch/nn/modules/linear.py#L93
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))   # Activación ReLU https://pytorch.org/docs/stable/nn.functional.html
        x = self.fc2(x)           # No activación si regresión, softmax si clasificación
        return x

# Instanciamos el modelo
model = MLP(input_dim=10, hidden_dim=32, output_dim=1)

# Definimos pérdida y optimizador
criterion = nn.MSELoss()               # Para regresión
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Ejemplo de datos ficticios
x = torch.randn(2000000, 10)   # 16 muestras, 10 features
y = torch.randn(2000000, 1)    # 16 etiquetas reales

start_time = time.time()
# Forward + Backward
model.train()
optimizer.zero_grad()
output = model(x)
loss = criterion(output, y)
loss.backward()
optimizer.step()
end_time = time.time()
elapsed = end_time - start_time

print("Pérdida:", loss.item())
print(f"Tiempo de entrenamiento: {elapsed:.6f} segundos")

Pérdida: 1.0366430282592773
Tiempo de entrenamiento: 1.229198 segundos


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# Seleccionamos el dispositivo: GPU si está disponible, sino CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

# Definimos la clase del modelo MLP
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instanciamos el modelo y lo movemos al dispositivo
model = MLP(input_dim=10, hidden_dim=32, output_dim=1).to(device)

# Función de pérdida y optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Generamos datos ficticios y los movemos al mismo dispositivo
x = torch.randn(2000000, 10).to(device)  # 16 muestras, 10 features
y = torch.randn(2000000, 1).to(device)
torch.cuda.synchronize()  # Antes de medir tiempo con GPU

start_time = time.time()

# Entrenamiento
model.train()
optimizer.zero_grad()
output = model(x)
loss = criterion(output, y)
loss.backward()
optimizer.step()

end_time = time.time()
elapsed = end_time - start_time

print("Pérdida:", loss.item())
print(f"Tiempo de entrenamiento: {elapsed:.6f} segundos")


Usando dispositivo: cuda
Pérdida: 1.130437970161438
Tiempo de entrenamiento: 0.687551 segundos
